In [2]:
import numpy as np #Library to handle data in vectorised manner
import pandas as pd #Library for data analysis
import json #Library to handle JSON files
from geopy.geocoders import Nominatim
GeoLocator = Nominatim(user_agent='application') #Convert an address into latitude and longitude
import requests #handle requests

#Matplotlib plotting modules
import matplotlib.cm as cm 
import matplotlib.colors as colors

#import k means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported')

Libraries imported


In [3]:
toronto_task1_csv= 'Toronto.TASK_1_df.csv'
toronto_task2_csv= 'Toronto.TASK_2_df.csv'

In [4]:
toronto_neighbourhoods = pd.read_csv(toronto_task2_csv)
toronto_neighbourhoods.shape
toronto_neighbourhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
address='Toronto, Ontario Canada'
geolocator = Nominatim(user_agent='application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinate of {}, are {} and {}".format(address,location.latitude,location.longitude))

The geographical coordinate of Toronto, Ontario Canada, are 43.6534817 and -79.3839347


In [9]:
#create a map of Toronto
map_toronto = folium.Map(location=[latitude,longitude],zoom_start=11)
#add markets to the map
for lat,lng, borough,neighbourhood in zip(toronto_neighbourhoods['Latitude'],toronto_neighbourhoods['Longitude'],
                                          toronto_neighbourhoods['Borough'],toronto_neighbourhoods['Neighbourhood']):
    label='{},{}'.format(neighbourhood,borough)
    label= folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=4,
    popup=label,
    color='blue',
    fill= True,
    fill_color='#87cefa',
    fill_opacity=0.5,
    parse_html=False
    ).add_to(map_toronto)

In [10]:
toronto_data= toronto_neighbourhoods[toronto_neighbourhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.shape

(39, 4)

In [11]:
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,East Toronto,Studio District,43.659526,-79.340923
4,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
for lat,lng,borough,neighbourhood in zip(toronto_data['Latitude'],toronto_data['Longitude'],
                                        toronto_data['Borough'],toronto_data['Neighbourhood']):
    label='{},{}'.format(neighbourhood,borough)
    label= folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=4,
    popup=label,
    color='blue',
    fill= True,
    fill_color='#87cefa',
    fill_opacity=0.5,
    parse_html=False
    ).add_to(map_toronto)

In [13]:
map_toronto

In [27]:
# Exploring Neighbourhood in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius = 500):
    #pass in Neighbourhood names, lat and long
    CLIENT_ID = 'M1MK3U3134ZR3BC4YKO30W0MSEKLQYVCUV1SN1QZMYIHSKSC' # your Foursquare ID
    CLIENT_SECRET = 'RJBJQRLLFTFUUZZIQVHZY4I1GSAOA5A0XQOOLZSRDVLP23UN' # your Foursquare Secret
    VERSION = '20210214'
    LIMIT = 100
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        #print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT 
        )
    #GET REQUEST 
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
        
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighbourhood',
                          'Neighbourhood Latitude',
                          'Neighbourhood Longitude',
                          'Venue',
                          'Venue Latitude',
                          'Venue Longitude',
                          'Venue Category']
    return(nearby_venues)

In [28]:
toronto_neighbourhoods= toronto_data
toronto_venues = getNearbyVenues(names=toronto_neighbourhoods['Neighbourhood'], 
                                 latitudes=toronto_neighbourhoods['Latitude'], 
                                 longitudes= toronto_neighbourhoods['Longitude'])

In [29]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [30]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,15,15,15,15,15,15
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,32,32,32,32,32,32


In [31]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 unique categories


In [32]:
#Analyse each Neighbourhood

#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix='',prefix_sep='')

#add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

# move Neighbourhood to first column
fixed_columns =[toronto_onehot.columns[-1]]+ list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]
toronto_onehot.shape

(1608, 229)

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385,0.015385


In [34]:
num_top_venues=5
for neigh in toronto_grouped['Neighbourhood']:
    print("-----"+ neigh + "-----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood']== neigh].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

-----Berczy Park-----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.07
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


-----Brockton, Parkdale Village, Exhibition Place-----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2       Nightclub  0.09
3     Coffee Shop  0.09
4       Pet Store  0.04


-----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto-----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3                Park  0.06
4          Comic Shop  0.06


-----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport-----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


-----Central Bay Street-----
                venue  freq
0         